In [2]:
import torch
from torch.distributions.multivariate_normal import MultivariateNormal

In [106]:
number_of_time_steps = 4
number_of_time_series = 4000

def obtain_rolling_correlations(number_of_time_steps = 4):
    rho_values = torch.linspace(0.,1.,number_of_time_steps)
    rho_values = torch.flip(rho_values,dims=(0,))
    all_rho_values = []
    for time_step in range(number_of_time_steps):
        column = torch.zeros(number_of_time_steps)
        rolled_rhos = torch.roll(rho_values,shifts=time_step)
        column[time_step:] = rolled_rhos[time_step:]
        all_rho_values.append(column)
    all_rho_values = torch.vstack(all_rho_values)
    return all_rho_values

def copy_upper_diagonal_values(tensor):
    # Create a sample 2D tensor (replace this with your own tensor)
    original_tensor = tensor
    # Get the upper triangular portion of the tensor (above the diagonal)
    upper_triangular = torch.triu(original_tensor, diagonal=1)
    # Create a mask for the diagonal and below
    mask = torch.tril(torch.ones_like(original_tensor), diagonal=0)
    # Fill the lower triangular portion with values from the upper triangular portion
    result_tensor = (upper_triangular + upper_triangular.transpose(0, 1))
    # Add the diagonal elements from the original tensor
    result_tensor += torch.diag_embed(torch.diag(original_tensor))
    return result_tensor


all_rho_values = obtain_rolling_correlations(number_of_time_steps)
covariance = copy_upper_diagonal_values(all_rho_values)
mu = torch.zeros(number_of_time_steps)
distribution_for_series = MultivariateNormal(mu,covariance)
sample = distribution_for_series.sample((number_of_time_series,))
rho = np.corrcoef(sample[:, 0], sample[:, 2])[0, 1]

In [109]:
rho

0.3471469871790427

In [105]:
covariance

tensor([[1.0000, 0.6667, 0.3333, 0.0000],
        [0.6667, 1.0000, 0.6667, 0.3333],
        [0.3333, 0.6667, 1.0000, 0.6667],
        [0.0000, 0.3333, 0.6667, 1.0000]])

In [81]:
covariance.T == covariance

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

MultivariateNormal(loc: torch.Size([4]), covariance_matrix: torch.Size([4, 4]))

In [96]:
import numpy as np

# Generate a sample from a bivariate Gaussian distribution
mean = [0, 0]
cov = [[1, 0.5], [0.5, 1]]
sample = np.random.multivariate_normal(mean, cov, size=4000)

# Calculate the correlation coefficient
rho = np.corrcoef(sample[:, 0], sample[:, 1])[0, 1]

print(f"The correlation coefficient is {rho:.2f}")


The correlation coefficient is 0.50
